In [ ]:
! pip install yargy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5930e8cda7d31cd5adf98944d718f68f39c19b51e99f34cf74c2c89e9efa6d11
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
from yargy import Parser
import pandas as pd

from yargy.predicates import gram, gte, lte
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, dictionary, eq, caseless



# инициализируем шаблон и правило для имени
Name = fact(
    'Name',
    ['first', "last"]
)

first = gram('Name').interpretation(Name.first.inflected())
last = gram('Surn').interpretation(Name.last.inflected())


NAME = rule( # составная часть, одна из тех, что пойдёшь в конечный fact Nmae
    first,
    last.optional()

).interpretation(
    Name
)




In [ ]:
list_city = pd.read_csv("/content/city.csv",  encoding='cp1251', sep = '";"', index_col = 0)

list_city['name"'] = list_city['name"'].apply(lambda x: x[:-1])
list_city = list_city['name"'].tolist()

/tmp/ipython-input-273784597.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  list_city = pd.read_csv("/content/city.csv",  encoding='cp1251', sep = '";"', index_col = 0)


True

In [ ]:
# инициализируем шаблон и правило для даты рождения



MONTHS = {
    'января','февраля','марта','апреля','мая','июня',
    'июля','августа','сентября','октября','ноября','декабря'
}
MONTH_NAME = dictionary(MONTHS)


MONTH = and_(
    gte(1),
    lte(12)
)
DAY = and_(
    gte(1),
    lte(31)
)
YEAR = and_(
    gte(1900),
    lte(2100)
)

YEAR_WORD = or_(
    caseless('год'),
    caseless('года'),
    caseless('году'),
    caseless('г')
)

DATE_YEAR_ONLY = rule(
    YEAR,
    YEAR_WORD.optional()
)


DATE = or_(
    rule(DAY, MONTH_NAME, YEAR),
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(YEAR, 'г', '.'),
    rule(DAY, '.', MONTH, '.', YEAR),
    rule(DAY, '/', MONTH, '/', YEAR),
    rule(MONTH_NAME, YEAR),
    rule(MONTH, '.', YEAR),
    rule(MONTH, '/', YEAR),
    rule(MONTH, '-', YEAR),
    rule(DAY, MONTH_NAME, YEAR, YEAR_WORD.optional()),
    DATE_YEAR_ONLY
).named('DATE')



BIRTH_DATE = rule(
    caseless('дата'),
    caseless('рождения')
)


BIRTH_PLACE = rule(
    caseless('место'),
    caseless('рождения')
)


BIRTH_YEAR = rule(
    caseless('год'),
    caseless('рождения')
)

BIRTH_ABBR = or_(
    rule(caseless('г'), eq('.'), caseless('р'), eq('.')),
    rule(caseless('гр'), eq('.')),
    rule(caseless('г'), caseless('р'))
)


FROM = rule(
    caseless('родом'),
    caseless('из')
)


NATIVE = morph_pipeline(['уроженец'])

BORN_ANCHOR = or_(
    rule(BORN_VERB, IN.optional()),
    BIRTH_DATE,
    BIRTH_PLACE,
    BIRTH_YEAR,
    BIRTH_ABBR,
    FROM,
    rule(NATIVE)
).named('BORN_ANCHOR')

born_anchor_parser = Parser(BORN_ANCHOR)

date_parser = Parser(DATE)

In [ ]:
# инициализируем шаблон и правило для места рождения

BirthPlace = fact('BirthPlace', ['place'])

IN = or_(eq('в'), eq('во'))

PLACE_PREFIX = or_(
    eq('город'), eq('г'), eq('г.'),
    eq('село'), eq('деревня'), eq('поселок'), eq('посёлок')
).optional()


GEO = dictionary(list_city)

PLACE = rule(
    IN.optional(),
    PLACE_PREFIX,
    GEO

).interpretation(BirthPlace.place)

place_parser = Parser(PLACE)

In [ ]:
def _first_match_in_window(parser, text, start, window):

    chunk = text[start:start+window]

    matches = list(parser.findall(chunk))

    if not matches:
        return None

    m = min(matches, key=lambda x: x.span.start)
    abs_span = (start + m.span.start, start + m.span.stop)

    return m, abs_span

def extract_birth_info(text, window_chars):

    anchors = list(born_anchor_parser.findall(text))

    if not anchors:
        return {"birth_date": None, "birth_place": None, "anchor_true": False}

    best = {"birth_date": None, "birth_place": None, 'anchor_true': True}

    best_score = -10**9

    for a in anchors:
        pos = a.span.stop

        d = _first_match_in_window(date_parser, text, pos, window_chars)
        p = _first_match_in_window(place_parser, text, pos, window_chars)

        score = 0
        if d:
            score += 10 - (d[1][0] - pos) * 0.01
        if p:
            score += 10 - (p[1][0] - pos) * 0.01


        if d and p:
            score += 5

        if score > best_score:
            best_score = score
            if d:
                best["birth_date"] = text[d[1][0]:d[1][1]]
            if p:
                best["birth_place"] = text[p[1][0]:p[1][1]]

    return best

In [ ]:
name_parser = Parser(NAME)

def extract_name_near_birth(text, left_window):

    anchors = list(born_anchor_parser.findall(text))
    if not anchors:
        return None

    a = anchors[0]
    left_start = max(0, a.span.start - left_window)
    left_chunk = text[left_start:a.span.start]

    ms = list(name_parser.findall(left_chunk))
    if not ms:
        return None

    m = max(ms, key=lambda x: x.span.stop)
    return left_chunk[m.span.start:m.span.stop]


def parse_person_by_birth(text):
    birth = extract_birth_info(text, 250)
    name = extract_name_near_birth(text, 200)
    return {
        "name": name,
        "birth_date": birth["birth_date"],
        "birth_place": birth["birth_place"],
        "anchor_true": birth["anchor_true"]
    }

In [ ]:
with open("/content/news.txt", "r") as f:
   lines_list = f.readlines()


In [ ]:
data = pd.DataFrame(columns = ["name", "birth_date", "birth_place", "anchor_true"])

for line in lines_list:
    parse_data = parse_person_by_birth(line)

    new_row = pd.DataFrame([{
    'name': parse_data['name'],
    'birth_date': parse_data['birth_date'],
    'birth_place': parse_data['birth_place'],
    'anchor_true': parse_data['anchor_true']}]
                           )

    data = pd.concat([data, new_row], ignore_index = True)

In [ ]:
data[data['anchor_true'] == True]

,name,birth_date,birth_place,anchor_true
41,Стадник,1971 году,Он,True
80,Жером,1968 году,он,True
93,Дэниел Эдельман,1920 году,Йорке,True
136,Кардашян,21 октября 1980 года,Голливуда,True
150,Владимира Маркина,None,по,True
205,Зинаиды Серебряковой,28 июня 1913 года,русской,True
223,Искандер,1990,Сухуми,True
266,Сахай,1984 году,он,True
273,В,None,None,True
343,Абдурашид Магомедов,феврале 2015,По,True


In [ ]:
parse_person_by_birth("Колин Дэвис родился в графстве Суррей в 1927 году")

{'name': 'Колин Дэвис', 'birth_date': None, 'birth_place': None}

In [ ]:
list(date_parser.findall("Иероним Карл Фридрих барон фон Мюнхгаузен родился 11 мая 1720 года в поместье Боденвердер недалеко от Ганновера"))

[]